### Notebook para treinar modelo de Entity Linkin

In [1]:
import numpy as np
import random

### Carregando dataset

In [2]:
text = np.load('../../Corpora/PetroNER-LinkedEntity/sentences.npy')
entities = np.load('../../Corpora/PetroNER-LinkedEntity/entities.npy')
classes = np.load('../../Corpora/PetroNER-LinkedEntity/classes.npy')
URI =  np.load('../../Corpora/PetroNER-LinkedEntity/URI.npy')
URIvec =  np.load('../../Corpora/PetroNER-LinkedEntity/URI_vectors.npy')

### **A separação entre treino e teste deverá ser refeita depois que a PUC entregar o PetroNER dividido entre trieno e teste**

Iremos separar o conjunto de treino, validação e teste baseado nas URI. Ou seja, as URI observadas na fase de treinamento do modelo não será usada para a validação e teste.

In [4]:
# Listando as URI referente às classes que aparecerão nos datasets de treino, validação e teste

# Lista de URI que não serão separadas entre treino, validação e teste 
# Incluímos manualmente as URI das principais classes
URI_not_split = ['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#basin',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#field',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#well',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#rock',
                 None]

# Fração das URI que não serão separadas entre treino, validação e teste 
fra_URI_not_split = 0.6

for U in set(URI):   
    if random.random() < fra_URI_not_split:
        URI_not_split.append(U)

URItre = []
URIval = []
URItes = []

# Separando as URI para treino (70%), validação (15%) e teste (15%)
for U in set(URI):   
    if U in URI_not_split:
        pass
    else:
        aleat = random.random()
        if aleat < 0.70:
            URItre.append(U)
        else:
            if aleat < 0.85:
                URIval.append(U)
            else:
                URItes.append(U)


In [7]:
# Separando os dataset de treino, validação e teste

# Listas de treino
text_treino = []
entities_treino = []
classes_treino = []
URI_treino = []
URIvec_treino = []

# Listas de validação
text_valid = []
entities_valid = []
classes_valid = []
URI_valid = []
URIvec_valid = []

# Listas de teste
text_teste = []
entities_teste = []
classes_teste = []
URI_teste = []
URIvec_teste = []

for n in range(len(URI)):
    # Verificando se a URI pertence a lista 'URI_not_split' distribuir usando a mesma proporção entre treino, validação e teste.
    if URI[n] in URI_not_split:
        aleat = random.random()
        if aleat < 0.70:
            text_treino.append(text[n])
            entities_treino.append(entities[n])
            classes_treino.append(classes[n])
            URI_treino.append(URI[n])
            URIvec_treino.append(URIvec[n])
            
        else:
            if aleat < 0.85:
                text_valid.append(text[n])
                entities_valid.append(entities[n])
                classes_valid.append(classes[n])
                URI_valid.append(URI[n])
                URIvec_valid.append(URIvec[n])
                
            else:
                text_teste.append(text[n])
                entities_teste.append(entities[n])
                classes_teste.append(classes[n])
                URI_teste.append(URI[n])
                URIvec_teste.append(URIvec[n])
                
    # Verificando se URI está na lista de treino
    if URI[n] in URItre:
        text_treino.append(text[n])
        entities_treino.append(entities[n])
        classes_treino.append(classes[n])
        URI_treino.append(URI[n])
        URIvec_treino.append(URIvec[n])
        
    # Verificando se URI está na lista de validação
    if URI[n] in URIval:
        text_valid.append(text[n])
        entities_valid.append(entities[n])
        classes_valid.append(classes[n])
        URI_valid.append(URI[n])
        URIvec_valid.append(URIvec[n])
        
    # Verificando se URI está na lista de teste
    if URI[n] in URItes:
        text_teste.append(text[n])
        entities_teste.append(entities[n])
        classes_teste.append(classes[n])
        URI_teste.append(URI[n])
        URIvec_teste.append(URIvec[n])
        
# Listas de treino
text_treino = np.array(text_treino)
entities_treino = np.array(entities_treino)
classes_treino = np.array(classes_treino)
URI_treino = np.array(URI_treino)
URIvec_treino = np.array(URIvec_treino)

# Listas de validação
text_valid = np.array(text_valid)
entities_valid = np.array(entities_valid)
classes_valid = np.array(classes_valid)
URI_valid = np.array(URI_valid)
URIvec_valid = np.array(URIvec_valid)

# Listas de teste
text_teste = np.array(text_teste)
entities_teste = np.array(entities_teste)
classes_teste = np.array(classes_teste)
URI_teste = np.array(URI_teste)
URIvec_teste = np.array(URIvec_teste)

In [9]:
print ('Treino')
print('Texto: ', len(text_treino))
print('Entidades: ', len(entities_treino))
print('Classes: ', len(classes_treino))
print('URI: ', len(URI_treino))
print('URIvec: :', len(URIvec_treino))

print ('\n Validação')
print('Texto: ', len(text_valid))
print('Entidades: ', len(entities_valid))
print('Classes: ', len(classes_valid))
print('URI: ', len(URI_valid))
print('URIvec: :', len(URIvec_valid))

print ('\n Teste')
print('Texto: ', len(text_teste))
print('Entidades: ', len(entities_teste))
print('Classes: ', len(classes_teste))
print('URI: ', len(URI_teste))
print('URIvec: :', len(URIvec_teste))

Treino
Texto:  7337
Entidades:  7337
Classes:  7337
URI:  7337
URIvec: : 7337

 Validação
Texto:  1394
Entidades:  1394
Classes:  1394
URI:  1394
URIvec: : 1394

 Teste
Texto:  1113
Entidades:  1113
Classes:  1113
URI:  1113
URIvec: : 1113
